<a href="https://colab.research.google.com/github/zaidme/qsite/blob/main/Copy_of_QSITE_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U classiq==0.48

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing instal

In [ ]:
import classiq
classiq.authenticate()

# High level quantum programming with Classiq

**Welcome to the Classiq Workshop for the QSITE 2024!**

The topic of this presentation is:

- **High level quantum programming with Classiq**

**[Classiq's documentation](https://docs.classiq.io/latest/)**




Additional resources you should use are
- The IDE of the classiq platform at [platform.classiq.io](https://platform.classiq.io)
- The [community Slack of Classiq](https://short.classiq.io/join-slack) - Classiq's team will answer any question you have over there, including implementation questions
- Our [GitHub repository](https://github.com/Classiq/classiq-library/tree/main)

  
**Good luck!**

## The beggining: Quantum functions and Quantum variables

The simplest quantum object is a single qubit, representing the values 0 or 1 when measured. Other types of quantum objects are stored on multiple qubits and represent numeric values or arrays of qubits.

Quantum objects are managed in Qmod using quantum variables. Variables are introduced into the scope of a quantum function through the declaration of arguments or the declaration of local variables.

A quantum variable establishes its reference to some object by explicitly initializing it. This is often done by passing it as the output argument of a function, such as `allocate()`. Once initialized, the state of the variable can be modified.

Quantum functions are functions that depends on quantum variables, and are defined as regular functions in python after using the decorator `@qfunc`.

<details>
<summary> Types of Quantum Variables </summary>
In Qmod there are 3 types of quantum variables:

1. `QBit` (`qbit`)
2. `QArray[QBit]` (`qbit[]`)
3. `QNum` (`qnum`)

(See also [Quantum Variables](https://nightly.docs.classiq.io/latest/classiq_101/classiq_concepts/design/quantum_variables_and_functions/))
</details>

In [ ]:
from classiq import *

## Example: The $|-\rangle$ state

The $|-\rangle$, also known as the "minus state", is important in quantum computing: It generates an equiprobable superposition of the computational basis, while having a negative phase between the states.

Using Classiq, you can build a function that prepare the minus state in four lines:

In [ ]:
@qfunc
def prepare_minus(q:QBit):
  X(q)
  H(q)

And synthesize it:

In [ ]:
@qfunc
def main(q:Output[QBit]):
  allocate(1,q)
  prepare_minus(q)

qmod = create_model(main)
qprog = synthesize(qmod)

You can use the IDE to analyze your quantum program using the `show` command:

In [ ]:
show(qprog)

## Before the next step: Guidelines for High-Level Functional Design with Classiq

**Some basic explanations about the high-level functional design with Classiq:**

* There should always be a `main` function - the model that captures your algorithm is described there

* The model is always generated out of the main function

* The model is sent to the synthesis engine (compiler) that return a quantum program which contains the quantum circuit

**Some basic guidelines about the modeling language (QMOD):**

1. Every quantum variable should be declared, either as an argument of a function e.g. `def prepare_minus(x: QBit)` or as a local variable within the function itself with `x = QBit('x')`

2. Some quantum variables need to be initialized with the `allocate` function. This is required in 2 cases:
* A variable is an argument of a function with the declaration `Output` like `def main(x: Output[QNum])`
* A variable that was declared within a function like `a = QNum('a')`

3. For the `main` function, you will always use `Output` for all variables. The `output` indicates that these quantum variables are not initialized outside the scope of the function.


<details>
<summary> Types of Initializations </summary>
There are a few ways to initialize a quantum variable:

1. With `allocate` or `allocate_num`
2. With `prepare_int`, `prepare_state` or `prepare_amplitudes`
3. As the result of a numeric operation `|=`
4. With the `bind` operation (`->` in native)
5. With any function that declares its quantum variable argument as `output`

</details>

# Tutorial 1: Preparing a state

Preparing specific states is mandatory in most quantum algorithms. With this in mind, it is important to know how to prepare the initial state of a quantum program using the function `prepare_state`.

Given a superposition of states with real, positive $\{c_i\}$:

$$ |\psi\rangle = \sum_{i=0}^{N} c_i |i\rangle $$

one can associate it to a probability list

$$P = [|c_0|^2,|c_1|^2,\dots,|c_{N}|^2]$$

Lets see an example using 3 qubits:


In [ ]:
prob_list = [0.1, 0.2, 0.3, 0, 0 , 0, 0, 0.4]
@qfunc
def main(q:Output[QArray[QBit]]):
  prepare_state(probabilities=prob_list, bound=0.01, out = q)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

You can use also the `inplace_prepare_state` function when you need to execute the quantum operation necessary to prepare the state in the middle of your quantum program.

## Another example: `inplace_prepare_state` and `prepare_state`

Now lets execute the operation necessary to prepare the state given by the probabilities

$$P = [0.625, 0, 0, 0, 0.125, 0.125, 0.125, 0]$$

in the state generated through `prob_list`:

In [ ]:
prob_list2 = [0.625, 0, 0, 0, 0.125, 0.125, 0.125, 0]
@qfunc
def main(q:Output[QArray[QBit]]):
  prepare_state(probabilities=prob_list, bound=0.01, out = q)
  inplace_prepare_state(probabilities=prob_list2, bound= 0.01, target=q)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

# Tutorial 2: Circuit optimization and arithmetic expressions.

Knowing what is the optimal number of gates or qubits in a quantum program are important informations that you can easily execute using the optimization tools in Classiq. Another important tool is the ability to realize arithmetic expressions.

To see the functionality of these tools, lets work on the problem of evaluating `y=x²+x+1` using a quantum program:


In [ ]:
@qfunc
def main(x:Output[QNum],y:Output[QNum]):
  #Initializing the variable 'x'
  allocate(3,x)
  #Preparing 'x' in a superposition of all possible numbers between 0 and 7
  apply_to_all(H,x)
  #Evaluating the arithmetic expression
  y |= x**2+x+1

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Optimization can be done by setting constraints in your circuit or defining wheter you want to optimize the depth (number of operations) or width (number of qubits) in your quantum program. Lets try to optimize the latter program in depth with a fixed width of 9 qubits:

In [ ]:
constraints = Constraints(optimization_parameter="depth", max_width=9)

qmod = set_constraints(qmod, constraints)
qprog = synthesize(qmod)
show(qprog)

# Your turn: Try to finish the following quantum program

Using the function `prepare_state`, initialize two quantum variables `x` and `y` in the state with probabilities `P = [0.25, 0.25, 0.25, 0.25]`. After that, evaluate the arithmetic expression

$$y = 0.5x+y+(y+x)^2$$

and set a quantum program optimized in depth, with a maximum width of `max_width=16`.

In [ ]:
probs = [0.25, 0.25, 0.25, 0.25]
@qfunc
def main(x:Output[QNum],y:Output[QNum],z:Output[QNum]):
  #TODO: Prepare 'x' and 'y' in a superposition of all possible numbers between 0 and 3

  #TODO: Evaluate the arithmetic expression
  z |=

qmod = create_model(main)

constraints = Constraints(optimization_parameter="depth", max_width=16)
qmod = set_constraints(qmod, constraints)
qprog = synthesize(qmod)
show(qprog)